In [1]:
from dotenv import load_dotenv 
from openai import OpenAI
load_dotenv()
client = OpenAI()



In [2]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "which city is China's Capital?"
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Function calls itself,  \nLayers of thought intertwine,  \nInfinite echoes.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [5]:
completion.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Function calls itself,  \nLayers of thought intertwine,  \nInfinite echoes.', refusal=None, role='assistant', function_call=None, tool_calls=None))